<a href="https://colab.research.google.com/github/75169/UMwfin/blob/Zadanie-2/Case2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pandas matplotlib

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
ratings_df = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies_df = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

In [5]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
merged_df = pd.merge(movies_df, ratings_df, on='movieId')

In [11]:
sci_fi_count = movies_df[movies_df['genres'].str.contains('Sci-Fi')].shape[0]
print(f"Liczba filmów Sci-Fi: {sci_fi_count}")

Liczba filmów Sci-Fi: 980


In [ ]:
comedy_ratings = merged_df[merged_df['genres'].str.contains('Comedy')]['rating']
print("Rozkład ocen dla filmów komediowych:")
print(comedy_ratings.describe())

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(comedy_ratings, bins=20, color='blue', alpha=0.7)
plt.title('Rozkład ocen komedii')
plt.xlabel('Ocena')
plt.ylabel('Liczba ocen')
plt.grid(True)
plt.show()

In [ ]:
action_ratings = merged_df[merged_df['genres'].str.contains('Action')]
average_action_rating = action_ratings['rating'].mean()
print(f"Średnia ocena filmów akcji: {average_action_rating:.2f}")

In [ ]:
min_number_of_ratings = 100
top_action_movies = action_ratings.groupby('title').agg({'rating': ['mean', 'count']})
top_action_movies = top_action_movies[top_action_movies[('rating', 'count')] >= min_number_of_ratings]
top_action_movies = top_action_movies.sort_values(by=[('rating', 'mean'), ('rating', 'count')], ascending=[False, False]).head(3)
print("Trzy najlepiej oceniane filmy akcji:")
print(top_action_movies)

In [ ]:
!pip install scikit-surprise

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import SVD, KNNWithMeans

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

svd = SVD()
knn_with_means = KNNWithMeans()

svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
knn_results = cross_validate(knn_with_means, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


# **Różnice między KNNWithMeans a standardowym KNN**
Algorytm KNNWithMeans różni się od standardowego KNN tym, że bierze pod uwagę średnią ocenę każdego użytkownika podczas obliczania podobieństw. Dzięki temu jest w stanie lepiej radzić sobie z osobistymi tendencjami oceniania, co może prowadzić do bardziej trafnych rekomendacji, szczególnie gdy użytkownicy mają różne skale oceniania.

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'k': [2, 3, 4, 5, 6]}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['RMSE', 'MAE'], cv=3)
gs.fit(data)

print(gs.best_params)
print(gs.best_score)


In [ ]:
from surprise import KNNWithMeans, Dataset, Reader
from surprise.model_selection import train_test_split

data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
model = KNNWithMeans()
model.fit(trainset)

def get_similar_movies(movie_title, k=10):
    movie_id = movies_df[movies_df['title'].str.contains(movie_title)].iloc[0]['movieId']
    neighbors = model.get_neighbors(movie_id, k=k)
    neighbor_movies = movies_df[movies_df['movieId'].isin(neighbors)]
    return neighbor_movies

similar_to_jumanji = get_similar_movies('Jumanji')
similar_to_flint = get_similar_movies('Flint')

print("Filmy podobne do 'Jumanji':\n", similar_to_jumanji)
print("Filmy podobne do 'Flint':\n", similar_to_flint)
